In [166]:
import pandas as pd
import os

In [ ]:
#data quality extract
products_data['ticker'] = products_data['ticker'].str.upper()


In [ ]:
len('3000')

In [542]:
class WisdomTreeDataPipeline:
    def __init__(self, file_path):
        self.file_path = file_path
        self.excel_file = pd.ExcelFile(file_path)
        self.products_table = self.extract_products()
        self.expense_ratios = self.extract_expense_ratios()

    def nav_format_and_convert_date(self, date_string:str):
        """
        Check if the first part of the date is >= 12, 
        then reformat and convert to datetime to y-m-d.
        """
        try:
            if "/" in date_string:
                date_part = date_string.split("/")
            else:
                date_part = date_string.split("-")       
         
            if len(date_part) == 3 and int(date_part[0]) > 12:
                formatted_date = f"{date_part[2]}-{date_part[1]}-{date_part[0]}"
    
            else:
                formatted_date = f"{date_part[2]}-{date_part[0]}-{date_part[1]}" 
    
            
            return pd.to_datetime(formatted_date, format='%Y-%m-%d', errors="coerce")
        except Exception as e:
            print(f"Error, nav_format_and_convert_date() failed: {str(e)}")
            return None
    
    def holdings_format_and_convert_date(self, date_string:str):
        """
        Check if the middle part of the date is >= 12, 
        then reformat and convert to datetime to y-m-d.
        """
        try:   
            if "/" in date_string:
                date_part = date_string.split("/")
            else:
                date_part = date_string.split("-")
                        
            if len(date_part) == 3 and int(date_part[1]) > 12:
                formatted_date = f"{date_part[0]}-{date_part[2]}-{date_part[1]}"
            else:
                formatted_date = date_string  # Keep original format if condition is not met
            
            return pd.to_datetime(formatted_date, format='%Y-%m-%d', errors="coerce")
        except Exception as e:
            print(f"Error, holdings_format_and_convert_date() failed: {str(e)}")
            return None

    def extract_products(self) -> pd.DataFrame:
        """
        1.Extracts WT Products table
        2.Renames columns.
        """
        try:
            products_df = pd.read_excel(self.excel_file, sheet_name="WT Products")
            products_df = products_df.rename(columns={
                "WT ID": "wisdom_tree_id",
                "Ticker": "ticker",
                "Product Name": "product_name"
            })    
            return products_df
        except Exception as e:
            print(f"Error, extract_products_data() failed: {str(e)}")
            return None
            
    def extract_expense_ratios(self) -> pd.DataFrame:
        """
        1. Extracts WT Expense Ratios table
        2. renames columns.
        """
        try:
            expense_df = pd.read_excel(self.excel_file, sheet_name="WT Expense Ratios")
            expense_df = expense_df.rename(columns={
                "Expense Ratio": "expense_ratio",
                "WT ID": "wisdom_tree_id"
            })
            return expense_df
        except Exception as e:
            print(f"Error, extract_expense_ratios() failed: {str(e)}")
            return None
                
    def extract_nav(self)->pd.DataFrame:
        """
        1. Extracts Net Asset Value Data from the Excel file 
        2. renames columns
        3. Converts date column to date type.
        """
        try:
            nav_df = pd.read_excel(self.excel_file, sheet_name="NAV Data")
            nav_df = nav_df.rename(columns={
            "WT ID": "wisdom_tree_id",
            "Date": "market_date",
            "NAV": "net_asset_value"
            })
            #If 'coerce', then invalid parsing will be set as NaT.
            nav_df["market_date"] = nav_df["market_date"].apply(self.nav_format_and_convert_date)
            # nav_df["market_date"] = pd.to_datetime(nav_df["market_date"], dayfirst=False, errors="coerce")

            # #reformat to yyyy-mm-dd
            # nav_df["market_date"] = pd.to_datetime(nav_df["market_date"], format='%Y-%m-%d', errors="coerce")
            
            nav_df["wisdom_tree_id"] = nav_df["wisdom_tree_id"].astype(int)

            # Adjust WCLD stock split (1-for-3 on 31 March 2024)
            nav_df.loc[(nav_df["wisdom_tree_id"] == 3105371) & (nav_df["market_date"] >= "2024-03-31"), "net_asset_value"] *= 3
            
            return nav_df
        except Exception as e:
            print(f"Error, extract_nav_data {str(e)}")
            return None
            # return  log.logMsg("Error", f"extract_nav_data() failed: {str(e)}")

    def process_client_holdings (self) -> pd.DataFrame:
        """
        1. Extracts multiple client holdings data from multiple sheets
        2. Combines client data into single table and adds start_date 
        and end_date for each quarter sheet to manage changing dimensions
        3. Adds id of tickers from products tables
        """
        try:
            client_holdings_list = []
            #Extract client ids
            client_ids_list = [sheet.split("_")[0] for sheet in self.excel_file.sheet_names if "client" in sheet.lower()]
            #Remove client ids duplicates and sort ascending 

            client_ids_list = list(dict.fromkeys(client_ids_list))
            client_ids_list.sort()

            #Loop by client 
            for client_id in client_ids_list:
                #Extract all sheets matching the client_id and sort ascending
                sheet_quarters_list = [sheet for sheet in self.excel_file.sheet_names if client_id in sheet]
                # quarters_list = [sheet_name.split("_")[1] for sheet_name in sheet_quarters_list]
                # print(quarters_list)
                sheet_quarters_list.sort()
                # print(sheet_quarters_list)
                #Loop thorugh sheet name to start extracting data 
                for sheet in sheet_quarters_list:
                    # Extract quarter date
                    sheet_quarter = sheet.split("_")[1]
                    # Extract all sheets of the same client id
                    client_sheet_df = pd.read_excel(self.excel_file, sheet_name=sheet)
                    #make all columns lower case
                    client_sheet_df.columns = map(str.lower, client_sheet_df.columns)
                    #make all acronyms upper case
                    client_sheet_df['ticker'] = client_sheet_df['ticker'].str.upper()
                    #unpivot table to create "month_date" column
                    client_unpivot_df = client_sheet_df.melt(id_vars=["ticker"], var_name="month_date", value_name="holdings")
                    #check for missing months
                    client_unpivot_df = self.fill_missing_months_holdings(client_id.lower(), sheet_quarter, client_unpivot_df)
                    
                    client_unpivot_df["client_id"] = client_id.lower()
                    client_unpivot_df["quarter_date"] = sheet_quarter
                    # client_unpivot_df["quarter_year"] = pd.DatetimeIndex(client_unpivot_df["quarter_date"]).year
                    # client_unpivot_df["quarter_month"] = pd.DatetimeIndex(client_unpivot_df["quarter_date"]).month
                    client_unpivot_df["start_date"] = sheet_quarter
                    client_unpivot_df["start_date"] = pd.to_datetime(client_unpivot_df["start_date"], format='%Y-%m-%d', errors="coerce")
                    client_unpivot_df["quarter_date"] = pd.to_datetime(client_unpivot_df["quarter_date"], format='%Y-%m-%d', errors="coerce")


                    #Create end_date column the mosth recent month_date 
                    #should have a null end_date else the most recent quarter
                    quarter_index = sheet_quarters_list.index(sheet)
                    if  quarter_index < len(sheet_quarters_list) - 1:
                        current_quarter = sheet_quarters_list[quarter_index].split("_")[1]
                        target_date = pd.Timestamp(current_quarter) - pd.DateOffset(months=8)
                        quarter_index += 1
                        # next_quarter_columns = pd.read_excel(self.excel_file, sheet_name=sheet_quarters_list[quarter_index], nrows=0).columns.tolist()
                        # next_quarter_columns.remove('ticker')
                        # next_quarter_columns = [pd.Timestamp(month_date) for month_date in next_quarter_columns]
                        # print(next_quarter_columns)
                        next_quarter =  pd.Timestamp(sheet_quarters_list[quarter_index].split("_")[1])
                        client_unpivot_df['end_date'] = client_unpivot_df['month_date'].apply(lambda month_date: next_quarter if month_date >= target_date else pd.NA)

                    else: 
                        client_unpivot_df["end_date"] = pd.NA

                    #If 'coerce', then invalid parsing will be set as NaT.
                    client_unpivot_df["end_date"] = pd.to_datetime(client_unpivot_df["end_date"], format='%Y-%m-%d', errors="coerce")
                    client_holdings_list.append(client_unpivot_df)

            # Join with products table to get wisdom_tree_id
            holdings_df = pd.concat(client_holdings_list, ignore_index=True)
            holdings_df = holdings_df.merge(self.products_table.drop(columns=["product_name"]), on="ticker", how="left")
            holdings_df.drop(columns=["ticker"])
            holdings_df["wisdom_tree_id"] = holdings_df["wisdom_tree_id"].astype(int)
            
            # Adjust for WCLD stock split (1:3 on 31 March 2024)
            holdings_df.loc[(holdings_df["wisdom_tree_id"] == 3105371) & (holdings_df["quarter_date"] >= "2024-03-31"), "holdings"] /= 3
            #remove columns
            holdings_df = holdings_df[["client_id", "quarter_date", "month_date", "wisdom_tree_id", "holdings", "start_date", "end_date"]]

            output_holdings_df = self.

            return holdings_df

        except Exception as e:
            print(f"Error, process_client_holdings() failed: {str(e)}")
            return None
            # return  log.logMsg("Error", f"process_client_holdings() failed: {str(e)}")

    def fill_missing_months_holdings(self, input_client_id:str, input_quarter_date: str, input_holdings_df: pd.DataFrame)->pd.DataFrame:
        """
        Checks for missing months and inserts row with holdings == 0
        """
        try:
            # Generate a full list of months from the given date going back 11 months
            all_months = pd.date_range(end=input_quarter_date, periods=12, freq='M')
            
            # Find the missing months
            input_holdings_df["month_date"] = input_holdings_df["month_date"].apply(self.holdings_format_and_convert_date)
            existing_months = input_holdings_df['month_date'].unique()
            missing_months = all_months.difference(existing_months)
            
            if len(missing_months) > 0:
                print(f"{input_client_id} | {input_quarter_date} | missing months:{list(missing_months)}")
                for name in input_holdings_df["ticker"].unique():
                    # Create rows for missing months with holdings set to 0
                    new_rows = pd.DataFrame({
                        'ticker': [name for month in range(len(missing_months))],
                        'month_date': missing_months,
                        'holdings': [0 for month in range(len(missing_months))]
                    })
                    # Append the new rows to the original DataFrame and sort by month_tdate
                    input_holdings_df = pd.concat([input_holdings_df, new_rows.astype(input_holdings_df.dtypes)], ignore_index=True).sort_values('month_date')

                    # print(input_holdings_df.dtypes)
                return input_holdings_df
    
            return input_holdings_df
        except Exception as e:
            print(f"Error, fill_missing_months_holdings() failed: {str(e)}")
            return None

    def fill_zero_holdings(self, holdings_df):
        """
        1. Replace holdings == 0 with the value from 
        the same month_date in the previous quarter
        2. Forward fills holdings with previous month 
        where that month does not exist on the previous quarter
        """
        try:
            holdings_df = holdings_df.sort_values(["client_id", "wisdom_tree_id", "quarter_date"])
            
            #Convert nulls and empty cells to 0
            holdings_df['holdings'] = holdings_df['holdings'].fillna(0)
            holdings_df['holdings'] = holdings_df['holdings'].replace('',0)
            holdings_df['holdings'] = holdings_df['holdings'].replace(' ',0)
            holdings_df["is_holdings_backfilled"] = (holdings_df["holdings"] == 0).astype(int)
            
            missing_holdings_mask = holdings_df["holdings"] == 0
            
            #Shift one to get previous quarter
            holdings_df.loc[missing_holdings_mask, "holdings"] = holdings_df.groupby(["client_id", "wisdom_tree_id", "month_date"])["holdings"].shift(1)
        
            #forward fills holdings previous month instead where 
            #month does not exist on the previous quarter
            if holdings_df["holdings"].isnull().values.any():
                holdings_df["holdings"] = holdings_df.groupby(["client_id", "wisdom_tree_id"])["holdings"].ffill()
        
            return holdings_df
        except Exception as e:
            print(f"Error, fill_missing_nav_dates() failed: {str(e)}")
            return None

    def fill_missing_nav_dates(self, nav_data: pd.DataFrame) -> pd.DataFrame:
        """
        Backfills NAV data of missing dates and NAV values
        """
        try:
            #Date range to get missing dates
            start_year = nav_data["market_date"].min().year
            end_year = nav_data["market_date"].max().year
            
            complete_dates = pd.date_range(start=f"{start_year}-01-01", 
                                       end=f"{end_year}-12-31", freq="D")
            
            # Create a new dataframe with completed date range
            # for each wisdom_tree_id
            unique_ids = nav_data["wisdom_tree_id"].unique()
            #Cartesian product of indexes using the product ids and full list of dates
            complete_index = pd.MultiIndex.from_product([unique_ids, complete_dates], names=["wisdom_tree_id", "market_date"])
        
            # Reindex NAV data to ensure all dates exist for each wisdom_tree_id
            nav_data = nav_data.set_index(["wisdom_tree_id", "market_date"]).reindex(complete_index)
        
            # Backward fill NAV values within each wisdom_tree_id
            nav_data = nav_data.sort_values(["market_date"]).reset_index()
            nav_data["is_nav_backfilled"] = nav_data["net_asset_value"].isna().astype(int)
            nav_data["net_asset_value"] = nav_data.groupby(["wisdom_tree_id"])["net_asset_value"].bfill()
        
            # Forward fill if any remaining nulls
            if nav_data["net_asset_value"].isnull().values.any():
                nav_data["net_asset_value"]  = nav_data.groupby(["wisdom_tree_id"])["net_asset_value"].ffill()
        
            return nav_data

        except Exception as e:
            print(f"Error, fill_missing_nav_dates() failed: {str(e)}")
            return None
                
    
    def transform_monthly_aum(self, input_holdings_df: pd.DataFrame, input_nav_df: pd.DataFrame) -> pd.DataFrame:
        """
        Estimates monthly AUM and daily revenue
        """
        try:

            input_holdings_df = input_holdings_df.sort_values(["client_id", "wisdom_tree_id", "month_date"])
            #Get the latest data of holdings
            holdings_latest_df = input_holdings_df[input_holdings_df["end_date"].isnull()]

            #Left join holdings table with with nav and expenses tables
            holdings_nav_df = holdings_latest_df.merge(input_nav_df,  left_on=["wisdom_tree_id", "month_date"], right_on=["wisdom_tree_id","market_date"], how="left")
            holdings_nav_expense_df = holdings_nav_df.merge(self.expense_ratios, on=["wisdom_tree_id"], how="left")

            
            # Adjust GGRA expense ratio change on 30 June 2024
            holdings_nav_expense_df.loc[(holdings_nav_expense_df['wisdom_tree_id'] == 1001656) & (holdings_nav_expense_df["market_date"] < pd.Timestamp('2024-06-30')), 'expense_ratio'] = .0028
            
            ##CALCULATIONS##
            #AUM Calculation
            holdings_nav_expense_df["assets_under_management"] = holdings_nav_expense_df["holdings"] * holdings_nav_expense_df["net_asset_value"]
            
            #Daily Revenue
            holdings_nav_expense_df["daily_revenue"] = holdings_nav_expense_df["assets_under_management"] * (holdings_nav_expense_df["expense_ratio"]/252)
           
            #Net Flow = share change (monthly holdings − previous month holdings) * monthly nav
            holdings_nav_expense_df["share_change"] = holdings_nav_expense_df["holdings"].diff()
            holdings_nav_expense_df["net_flow"] = holdings_nav_expense_df["share_change"] * holdings_nav_expense_df["net_asset_value"]
            
            #Market Movement = AUM currrent month - AUM at previous month-end−Net Flow
            holdings_nav_expense_df["market_movement"] = holdings_nav_expense_df["assets_under_management"].diff() - holdings_nav_expense_df["net_flow"]

            holdings_nav_expense_df = holdings_nav_expense_df[["client_id", "month_date", "wisdom_tree_id", "is_holdings_backfilled", "holdings", "is_nav_backfilled", "net_asset_value", "assets_under_management", "daily_revenue", "share_change", "net_flow", "market_movement"]]
            holdings_nav_expense_df.sort_values("month_date")
            
            #Clean oldest month for each client make null 
            wrong_values_mask = merged_df["month_date"] == merged_df["month_date"].min()
            holdings_nav_expense_df.loc[wrong_values_mask, "share_change"] = pd.NA
            holdings_nav_expense_df.loc[wrong_values_mask, "net_flow"] = pd.NA
            holdings_nav_expense_df.loc[wrong_values_mask, "market_movement"] = pd.NA
            return holdings_nav_expense_df
        except Exception as e:
            print(f"Error, transform_monthly_aum() failed: {str(e)}")
            return None
                


In [ ]:
def format_and_convert_date(date_string:str):
    """
    Check if the middle part of the date is >= 12, 
    then reformat and convert to datetime to y-m-d.
        """
    if "/" in date_string:
        date_part = date_string.split("/")
    else:
        date_part = date_string.split("-")

    if len(date_part) == 3 and int(date_part[1]) > 12:
        formatted_date = f"{date_part[0]}-{date_part[2]}-{date_part[1]}"
    else:
        formatted_date = date_string  # Keep original format if condition is not met
    
    return pd.to_datetime(formatted_date, format='%Y-%m-%d', errors="coerce")


In [192]:
import pandas as pd

# Sample DataFrame
data = {
    'month_date': pd.to_datetime([
        '2023-12-31', '2024-11-30', '2025-01-31', '2026-06-30'
    ])
}

df = pd.DataFrame(data)

# Add a new column with the specified condition
df['new_date'] = df['month_date'].apply(lambda x: pd.Timestamp('2024-12-31') if x >= pd.Timestamp('2024-12-31') else pd.NaT)

# Display the resulting DataFrame
df.dtypes

month_date    datetime64[ns]
new_date      datetime64[ns]
dtype: object

In [208]:
target_date = pd.Timestamp('2023-12-31') - pd.DateOffset(months=8)
target_date

Timestamp('2023-04-30 00:00:00')

In [543]:
file_path = "HistoricalClientHoldings.xlsx"
etl_pipeline = WisdomTreeDataPipeline(file_path)

In [544]:
expense_df = etl_pipeline.extract_expense_ratios()
nav_df = etl_pipeline.extract_nav()
holdings_df = etl_pipeline.process_client_holdings()

client1 | 2024-12-31 | missing months:[Timestamp('2024-12-31 00:00:00')]
client2 | 2024-12-31 | missing months:[Timestamp('2024-12-31 00:00:00')]
client3 | 2024-12-31 | missing months:[Timestamp('2024-12-31 00:00:00')]
client4 | 2024-12-31 | missing months:[Timestamp('2024-12-31 00:00:00')]
client5 | 2024-12-31 | missing months:[Timestamp('2024-12-31 00:00:00')]


In [549]:
nav_df_backfilled = fill_missing_nav_dates(nav_df)
holdings_cleaned = fill_zero_holdings(holdings_df)

In [551]:
holdings_cleaned.head()

,client_id,quarter_date,month_date,wisdom_tree_id,holdings,start_date,end_date,is_holdings_backfilled
1,client1,2023-12-31,2023-01-31,1001310,169624467.00000000,2023-12-31,NaT,0
5,client1,2023-12-31,2023-02-28,1001310,146680349.28479511,2023-12-31,NaT,0
9,client1,2023-12-31,2023-03-31,1001310,143811899.61335421,2023-12-31,NaT,0
13,client1,2023-12-31,2023-04-30,1001310,142820273.73600101,2023-12-31,2024-03-31,0
17,client1,2023-12-31,2023-05-31,1001310,140217663.79066101,2023-12-31,2024-03-31,0


In [552]:
merged_df = etl_pipeline.transform_monthly_aum(holdings_cleaned, nav_df_backfilled)

In [ ]:
mer

In [506]:
merged_df["share_change"] = merged_df["holdings"].diff()
merged_df["net_flow"] = merged_df["share_change"] * merged_df["net_asset_value"]
merged_df["market_movement"] = merged_df["assets_under_management"].diff() - merged_df["net_flow"]


In [508]:
merged_df["market_movement"] = merged_df["assets_under_management"].diff() - merged_df["net_flow"]


In [526]:
wrong_values_mask = merged_df["month_date"] == merged_df["month_date"].min()

merged_df.loc[wrong_values_mask, "share_change"] = pd.NA
merged_df.loc[wrong_values_mask, "net_flow"] = pd.NA
merged_df.loc[wrong_values_mask, "market_movement"] = pd.NA




In [511]:
# merged_df.head(40) #[(merged_df["month_date"]== merged_df["month_date"].min() ) & (merged_df["client_id"] == "client1")]

In [548]:
merged_df[merged_df["net_flow"] < 0].head(30)

,client_id,month_date,wisdom_tree_id,is_holdings_backfilled,holdings,is_nav_backfilled,net_asset_value,assets_under_management,daily_revenue,share_change,net_flow,market_movement
1,client1,2023-02-28,1001310,0,146680349.28479511,0,8.14186522,1194251634.64583063,23221.55956256,-22944117.71520489,-186807914.08321387,-54244494.10849279
2,client1,2023-03-31,1001310,0,143811899.61335421,1,9.22116777,1326113653.78674412,25785.54326808,-2868449.67144090,-26450455.66274303,158312474.80365652
3,client1,2023-04-30,1001310,0,142820273.73600101,1,8.40370676,1200219700.49685907,23337.60528744,-991625.87735319,-8333333.09332381,-117560620.19656123
4,client1,2023-05-31,1001310,0,140217663.79066101,0,8.27375359,1160126399.16403484,22558.01331708,-2602609.94534001,-21533353.37852711,-18559947.95429712
5,client1,2023-06-30,1001310,0,139349164.32483131,0,9.30296585,1296360517.52903605,25207.01006306,-868499.46582970,-8079620.87502848,144313739.24002969
6,client1,2023-07-31,1001310,0,130778900.22731370,0,8.82521956,1154152508.17109156,22441.85432555,-8570264.09751761,-75634462.33792956,-66573547.02001493
7,client1,2023-08-31,1001310,0,128158260.99542800,0,9.79472190,1255274526.17708707,24408.11578678,-2620639.23188570,-25668432.48757694,126790450.49357246
8,client1,2023-09-30,1001310,0,116809439.71560170,1,10.08637170,1178183427.36854935,22909.12219883,-11348821.27982630,-114468429.81708930,37377331.00855158
9,client1,2023-10-31,1001310,0,111740243.85424210,0,9.95452952,1112321555.77012801,21628.47469553,-5069195.86135960,-50461459.83327328,-15400411.76514805
11,client1,2023-12-31,1001310,0,99828143.54808500,1,10.33206853,1031431220.56220508,20055.60706649,-12829899.47348699,-132559400.61887416,18556069.86426172


In [ ]:
client_sheet_df = pd.read_excel(file_path, sheet_name="Client1_2023-12-31")
#make all columns lower case
client_sheet_df.columns = map(str.lower, client_sheet_df.columns)

#make all acronyms upper case
client_sheet_df['ticker'] = client_sheet_df['ticker'].str.upper()
client_unpivot_df = client_sheet_df.melt(id_vars=["ticker"], var_name="month_date", value_name="holdings")
# client_unpivot_df["month_date"] = pd.to_datetime(client_unpivot_df["month_date"], errors="coerce").dt.strftime('%Y-%m-%d')
client_unpivot_df["quarter_date"] = "2024-03-31"
client_unpivot_df["quarter_date"] = pd.to_datetime(client_unpivot_df["quarter_date"], format='%Y-%m-%d', errors="coerce")
client_unpivot_df["month_date"] = client_unpivot_df["month_date"].apply(format_and_convert_date)
month_dates_list =  client_unpivot_df["month_date"].unique()
# client_unpivot_df["client_id"] = client_id
# client_unpivot_df["quarter_date"] = sheet_quarter
# client_unpivot_df["quarter_year"] = pd.DatetimeIndex(client_unpivot_df["quarter_date"]).year
# client_unpivot_df["quarter_month"] = pd.DatetimeIndex(client_unpivot_df["quarter_date"]).month
# client_unpivot_df["start_date"] = sheet_quarter


In [77]:
def add_missing_months_holdings(input_quarter_date: str, input_holdings_df: pd.DataFrame)->pd.DataFrame:
    """
    Checks for missing months and inserts row with holdings == 0
    """
    try:

        # Generate a full list of months from the given date going back 11 months
        all_months = pd.date_range(end=input_quarter_date, periods=12, freq='M')
        print(all_months)
        # Find the missing months
        existing_months = input_holdings_df['month_date'].unique()
        missing_months = all_months.difference(existing_months)
        print(missing_months)
    
        for name in holdings_df["wisdom_tree_id"].unique():
            
            # Create rows for missing months with holdings set to 0
            new_rows = pd.DataFrame({
                'wisdom_tree_id': [name for month in range(len(missing_months))],
                'month_date': missing_months,
                'holdings': [0 for month in range(len(missing_months))]
            })
            # Append the new rows to the original DataFrame and sort by month_tdate
            input_holdings_df = pd.concat([input_holdings_df, new_rows.astype(input_holdings_df.dtypes)], ignore_index=True).sort_values('month_date')
    
        return input_holdings_df

    except Exception as e:
        print(f"Error, add_missing_months_holdings() failed: {str(e)}")
        return None
        # return  log.logMsg("Error", f"process_client_holdings() failed: {str(e)}")

# # Display the resulting DataFrame
# import ace_tools as tools; tools.display_dataframe_to_user(name="Complete DataFrame with Missing Months", dataframe=df_complete)


In [ ]:
# holdings_df["quarter_date"].unique()

In [7]:

holdings_test = holdings_df[(holdings_df["quarter_date"] == "2024-12-31") & (holdings_df["month_date"] != "2024-03-31") & (holdings_df["client_id"]  == "client1")] 



In [8]:
holdings_test = holdings_test[["wisdom_tree_id","month_date","holdings"]]

In [11]:
# holdings_test

In [ ]:
# holdings_added[holdings_added["month_date"] == "2024-12-31"]

In [316]:
def fill_missing_nav_dates(nav_data: pd.DataFrame) -> pd.DataFrame:
    """
    Backfills NAV data of missing dates and NAV values
    """
    try:
        #Date range to get missing dates
        start_year = nav_data["market_date"].min().year
        end_year = nav_data["market_date"].max().year
        
        complete_dates = pd.date_range(start=f"{start_year}-01-01", 
                                   end=f"{end_year}-12-31", freq="D")
        
        # Create a new dataframe with completed date range
        # for each wisdom_tree_id
        unique_ids = nav_data["wisdom_tree_id"].unique()
        complete_index = pd.MultiIndex.from_product([unique_ids, complete_dates], names=["wisdom_tree_id", "market_date"])
    
        # Reindex NAV data to ensure all dates exist for each wisdom_tree_id
        nav_data = nav_data.set_index(["wisdom_tree_id", "market_date"]).reindex(complete_index)
    
        # Backward fill NAV values within each wisdom_tree_id
        nav_data = nav_data.sort_values(["market_date"]).reset_index()
        nav_data["is_nav_backfilled"] = nav_data["net_asset_value"].isna().astype(int)
        nav_data["net_asset_value"] = nav_data.groupby(["wisdom_tree_id"])["net_asset_value"].bfill()
    
        # Forward fill if any remaining nulls
        if nav_data["net_asset_value"].isnull().values.any():
            nav_data["net_asset_value"]  = nav_data.groupby(["wisdom_tree_id"])["net_asset_value"].ffill()
    
        return nav_data

    except Exception as e:
        print(f"Error, fill_missing_nav_dates() failed: {str(e)}")
        return None

In [407]:
def fill_zero_holdings(holdings_df):
    """
    1. Replace holdings == 0 with the value from 
    the same month_date in the previous quarter
    2. Forward fills holdings with previous month 
    where that month does not exist on the previous quarter
    """
    try:
        holdings_df = holdings_df.sort_values(["client_id", "wisdom_tree_id", "quarter_date"])
        
        #Convert nulls and empty cells to 0
        holdings_df['holdings'] = holdings_df['holdings'].fillna(0)
        holdings_df['holdings'] = holdings_df['holdings'].replace('',0)
        holdings_df['holdings'] = holdings_df['holdings'].replace(' ',0)
        holdings_df["is_holdings_backfilled"] = (holdings_df["holdings"] == 0).astype(int)
        
        missing_holdings_mask = holdings_df["holdings"] == 0
        
        #Shift one to get previous quarter
        holdings_df.loc[missing_holdings_mask, "holdings"] = holdings_df.groupby(["client_id", "wisdom_tree_id", "month_date"])["holdings"].shift(1)
    
        #forward fills holdings previous month instead where 
        #month does not exist on the previous quarter
        if holdings_df["holdings"].isnull().values.any():
            holdings_df["holdings"] = holdings_df.groupby(["client_id", "wisdom_tree_id"])["holdings"].ffill()
    
        return holdings_df
    except Exception as e:
        print(f"Error, fill_missing_nav_dates() failed: {str(e)}")
        return None

In [ ]:
holdings_added = add_missing_months_holdings("2024-12-31", holdings_test)

In [ ]:
# holdings_df[(holdings_df["quarter_date"] == "2024-03-31") & (holdings_df["month_date"] != "2024-03-31") & (holdings_df["client_id"]  == "client1")] 

In [ ]:
from datetime import datetime
list(month_dates_list).index(datetime.strptime("2023-03-31", "%Y-%m-%d"))

In [ ]:
client_unpivot_df[(holdings_df["month_date"].isnull())]

In [ ]:
pd.options.display.float_format = '{:,.8f}'.format

In [ ]:
# Adjust GGRA expense ratio change on 30 June 2024
expense_data.loc[(expense_data["wisdom_tree_id"] == 1001656) & (expense_data["quarter_date"] >= "2024-06-30"), "expense_ratio"] = 0.38
expense_data.loc[(expense_data["wisdom_tree_id"] == 1001656) & (expense_data["quarter_date"] < "2024-06-30"), "expense_ratio"] = 0.28

In [ ]:
# quarter_dates_list = get_quarter_dates('2024-01-06')

In [346]:
nav_df_backfilled[nav_df_backfilled["net_asset_value"].isnull()]

,wisdom_tree_id,market_date,net_asset_value,is_nav_backfilled


In [345]:
# holdings_cleaned[(holdings_cleaned["client_id"]  == "client1") & (holdings_cleaned["end_date"].isnull()) & (holdings_cleaned["wisdom_tree_id"]  == 1001310)]

In [ ]:
# nav_df_backfilled[nav_df_backfilled["is_backfilled"] == 1]

In [ ]:
# for i in nav_df_backfilled["market_date"].unique():
#     print(i)

In [401]:
merged_navdf = nav_df_backfilled.merge(nav_df,  left_on=["wisdom_tree_id", "market_date"], right_on=["wisdom_tree_id","market_date"], how="left")

In [ ]:
# merged_navdf[(merged_navdf["market_date"] >= '2023-01-01') & (merged_navdf["market_date"] <= '2023-01-31') & (merged_navdf["wisdom_tree_id"] == 1001513)]

In [351]:
# merged_df[(merged_df["quarter_date"] == "2023-12-31") & (merged_df["client_id"]  == "Client1")] #& (holdings_df["end_date"].isnull())]

In [352]:
# holdings_cleaned[(holdings_cleaned["month_date"] >= "2024-03-31") & (holdings_cleaned["client_id"]  == "client1") &(holdings_cleaned["wisdom_tree_id"]  == 3105371)] 

In [ ]:
holdings_cleaned[holdings_cleaned["is_holdings_backfilled"] == 1]

In [ ]:
holdings_cleaned[(holdings_cleaned["month_date"] == "2024-11-30") & (holdings_cleaned["quarter_date"] == "2024-12-31") ]

In [ ]:
# null_mask = merged_df[[.isnull().any(axis=1)
# null_rows = merged_df[null_mask]
# null_rows

In [ ]:
# holdings_df[(holdings_df["quarter_date"] == "2024-09-30") & (holdings_df["client_id"]  == "Client2")] #& (holdings_df["end_date"].isnull())]

In [375]:
pd.options.display.float_format = '{:.8f}'.format


In [ ]:
nav_df[nav_df["quarter_date"] == "2023-02-28"]

In [359]:
nav_df_backfilled.columns

Index(['wisdom_tree_id', 'market_date', 'net_asset_value',
       'is_nav_backfilled'],
      dtype='object')

In [456]:
filtered_df = merged_df[(merged_df["client_id"]  == "client1")  & (merged_df["wisdom_tree_id"]  == 1001656)]

filtered_d fnav_data["market_date"].min().year

# filtered_df[["assets_under_management","daily_revenue"]]

,client_id,month_date,wisdom_tree_id,is_holdings_backfilled,holdings,is_nav_backfilled,net_asset_value,assets_under_management,daily_revenue
48,client1,2023-01-31,1001656,0,614252731.00000000,0,37.01238261,22734957100.95402145,252610.63445504
49,client1,2023-02-28,1001656,0,553567245.86088812,0,35.33076946,19557956744.46055603,217310.63049401
50,client1,2023-03-31,1001656,0,512732695.57045442,0,37.30624426,19128131179.97111511,212534.79088857
51,client1,2023-04-30,1001656,0,461935577.87212151,1,37.23104853,17198345915.82686615,191092.73239808
52,client1,2023-05-31,1001656,0,428789724.94600189,0,36.69084689,15732658146.54609108,174807.31273940
53,client1,2023-06-30,1001656,0,423442298.36570472,0,35.27243713,14935841845.46616364,165953.79828296
54,client1,2023-07-31,1001656,0,400054826.23287958,0,38.84045728,15538312388.24456406,172647.91542494
55,client1,2023-08-31,1001656,0,384510537.39772189,0,37.37232585,14370053095.18013191,159667.25661311
56,client1,2023-09-30,1001656,0,384196022.78067178,1,33.33220729,12806101471.96550751,142290.01635517
57,client1,2023-10-31,1001656,0,338148711.61399359,0,27.18854038,9193769901.10413551,102152.99890116


In [ ]:
# # Example Usage
# file_path = "HistoricalClientHoldings.xlsx"
# etl_pipeline = WisdomTreeDataPipeline(file_path)
# processed_data = etl_pipeline.run_etl()

# # Display final processed DataFrame
# import ace_tools as tools
# tools.display_dataframe_to_user(name="Processed WisdomTree Data", dataframe=processed_data)

In [246]:
from datetime import datetime
from datetime import datetime, date, timedelta


def get_quarter_dates(input_date: str = None)->list:
    """
    Returns the list of quarters given a date or todays date
    """
    try:
        if input_date is None:
            input_date = date.today()
        else:
            input_date = datetime.strptime(input_date, "%Y-%m-%d").date()
        year = input_date.year
        prev_year = year - 1
    
        # Define quarter-end date logic based on the given date
        month_day = int(input_date.strftime("%m%d"))
        
        if month_day < 331:
            quarter_dates = [f"{prev_year}-12-31", f"{year}-03-31", f"{prev_year}-06-30", f"{prev_year}-09-30", f"{year}-12-31"]
        elif month_day < 630:
            quarter_dates = [f"{prev_year}-03-31", f"{prev_year}-06-30", f"{prev_year}-09-30", f"{prev_year}-12-31", f"{year}-03-31"]
        elif month_day < 930:
            quarter_dates = [f"{prev_year}-06-30", f"{prev_year}-09-30", f"{prev_year}-12-31", f"{year}-03-31", f"{year}-06-30"]
        elif month_day < 1231:
            quarter_dates = [f"{prev_year}-09-30", f"{prev_year}-12-31", f"{year}-03-31", f"{year}-06-30", f"{year}-09-30",]
        else:
            return "get_quarter_dates() error"
    
        return quarter_dates
    except Exception as e:
        print(f"Error, get_quarter_dates() failed: {str(e)}")
        return None

# Example usage

date_str = '2024-11-12'
quarter_dates_list = get_quarter_dates(date_str)
print(quarter_dates_list)



['2023-09-30', '2023-12-31', '2024-03-31', '2024-06-30', '2024-09-30']
